# Importing Libraries

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
from numpy.random import randn
import numpy as np
from scipy import stats
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t
from termcolor import colored

In [4]:
# function for calculating the t-test for two independent samples
def independent_ttest(data1, data2, alpha):
 # calculate means
 mean1, mean2 = mean(data1), mean(data2)
 # calculate standard errors
 se1, se2 = sem(data1), sem(data2)
 # standard error on the difference between the samples
 sed = sqrt(se1**2.0 + se2**2.0)
 # calculate the t statistic
 t_stat = (mean1 - mean2) / sed
 # degrees of freedom
 df = len(data1) + len(data2) - 2
 # calculate the critical value
 cv = t.ppf(1.0 - alpha, df)
 # calculate the p-value
 p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
 # return everything
 return t_stat, df, cv, p

## Toy Samples

In [5]:
mu2, sigma2 = 25, 5 # mean and standard deviation
s2 = np.random.normal(mu2, sigma2, 5000)
mu, sigma = 30, 5 # mean and standard deviation
s = np.random.normal(mu, sigma, 5000)
zz = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]

# Population

In [6]:
df = pd.read_excel('DBGeneticExpressionCarcass.xlsx')
df['Group'] = df['Amostra'].apply(lambda x: x[0])
df['Day'] = df['Amostra'].apply(lambda x: int(x[1]) if x[1] != 'u' else 0)
df.head()

,Amostra,RPL32,VGR,VG,Group,Day
0,A1-1,19.731661,33.665260,25.279843,A,1
1,A1-1,19.851553,32.612957,25.306593,A,1
2,A1-2,19.448266,27.550795,24.992273,A,1
3,A1-2,19.733913,27.364021,24.987568,A,1
4,A1-3,18.480774,28.844547,24.236149,A,1


In [7]:
df[df['VG'].isna()]

,Amostra,RPL32,VGR,VG,Group,Day


In [8]:
df[df['Amostra'] == 'Pu-1']['VG'].mean() 

34.08478205348986

In [9]:
# df[df['Amostra'] == 'Pu-1']['VG'] = df[df['Amostra'] == 'Pu-1']['VG'].mean() 

In [10]:
df.describe()

,RPL32,VGR,VG,Day
count,78.000000,78.000000,78.000000,78.000000
mean,19.876526,29.664344,25.870927,4.000000
std,1.014678,2.232465,3.226914,3.006486
min,18.463029,24.188826,21.891192,0.000000
25%,19.096639,27.913504,23.845645,1.000000
50%,19.626455,29.592190,25.584168,4.000000
75%,20.647191,31.281618,26.415266,8.000000
max,22.333588,34.734440,35.986200,8.000000


In [11]:
df_mean = df # df.groupby(['Amostra', 'Group']).mean().reset_index()

In [12]:
df_mean

,Amostra,RPL32,VGR,VG,Group,Day
0,A1-1,19.731661,33.665260,25.279843,A,1
1,A1-1,19.851553,32.612957,25.306593,A,1
2,A1-2,19.448266,27.550795,24.992273,A,1
3,A1-2,19.733913,27.364021,24.987568,A,1
4,A1-3,18.480774,28.844547,24.236149,A,1
...,...,...,...,...,...,...
73,D8-1,21.674982,32.221210,27.392739,D,8
74,D8-2,20.920304,32.135807,25.942998,D,8
75,D8-2,21.002150,31.960926,26.077807,D,8
76,D8-3,22.317273,30.891573,30.746798,D,8


Each distribution is a total of eggs layed during 10 days for each female individual

In [13]:
df_mean['DCT_VG'] =  df_mean['VG'] - df_mean['RPL32']
df_mean['DCT_VGR'] =  df_mean['VGR'] - df_mean['RPL32']
df_mean.head()

,Amostra,RPL32,VGR,VG,Group,Day,DCT_VG,DCT_VGR
0,A1-1,19.731661,33.665260,25.279843,A,1,5.548182,13.933599
1,A1-1,19.851553,32.612957,25.306593,A,1,5.455040,12.761404
2,A1-2,19.448266,27.550795,24.992273,A,1,5.544006,8.102528
3,A1-2,19.733913,27.364021,24.987568,A,1,5.253656,7.630108
4,A1-3,18.480774,28.844547,24.236149,A,1,5.755375,10.363774


In [14]:
df_mean['DDCT_VG'] =  df_mean['DCT_VG'] - df_mean['DCT_VG'].min() 
df_mean['DDCT_VGR'] =  df_mean['DCT_VGR'] - df_mean['DCT_VGR'].min()
df_mean.head()

,Amostra,RPL32,VGR,VG,Group,Day,DCT_VG,DCT_VGR,DDCT_VG,DDCT_VGR
0,A1-1,19.731661,33.665260,25.279843,A,1,5.548182,13.933599,2.652499,8.723828
1,A1-1,19.851553,32.612957,25.306593,A,1,5.455040,12.761404,2.559357,7.551634
2,A1-2,19.448266,27.550795,24.992273,A,1,5.544006,8.102528,2.648323,2.892758
3,A1-2,19.733913,27.364021,24.987568,A,1,5.253656,7.630108,2.357972,2.420338
4,A1-3,18.480774,28.844547,24.236149,A,1,5.755375,10.363774,2.859692,5.154003


In [15]:
df_mean['EXP_DDCT_VG'] =  df_mean['DDCT_VG'].apply(lambda x: 2**(-x)) 
df_mean['EXP_DDCT_VGR'] =  df_mean['DDCT_VGR'].apply(lambda x: 2**(-x)) 
df_mean.head()

,Amostra,RPL32,VGR,VG,Group,Day,DCT_VG,DCT_VGR,DDCT_VG,DDCT_VGR,EXP_DDCT_VG,EXP_DDCT_VGR
0,A1-1,19.731661,33.665260,25.279843,A,1,5.548182,13.933599,2.652499,8.723828,0.159044,0.002365
1,A1-1,19.851553,32.612957,25.306593,A,1,5.455040,12.761404,2.559357,7.551634,0.169651,0.005330
2,A1-2,19.448266,27.550795,24.992273,A,1,5.544006,8.102528,2.648323,2.892758,0.159505,0.134646
3,A1-2,19.733913,27.364021,24.987568,A,1,5.253656,7.630108,2.357972,2.420338,0.195065,0.186812
4,A1-3,18.480774,28.844547,24.236149,A,1,5.755375,10.363774,2.859692,5.154003,0.137768,0.028086


## Sanity Test

In [16]:
def interpret_ttest(a, b, alpha=0.05):
    t_stat, df, cv, p = independent_ttest(a, b, alpha)
    print('t=%.3f, df=%d, cv=%.5f, p=%.8f' % (t_stat, df, cv, p))
#     # interpret via critical value
#     if abs(t_stat) <= cv:
#         print('Accept null hypothesis (Same distributions).')
#     else:
#         print('Reject the null hypothesis (Different distributions).')
    # interpret via p-value
    if p > alpha:
        print(colored('Accept null hypothesis (Same distributions).', 'green'))
    else:
        print(colored('Reject the null hypothesis (Different distributions).', 'red'))

In [17]:
interpret_ttest(s,s2)

t=49.708, df=9998, cv=1.64501, p=0.00000000
Reject the null hypothesis (Different distributions).


In [18]:
def interpret_kwtest(a, b, alpha = 0.05):
    stat, p = stats.kruskal(a, b)
    print('Statistics=%.3f, p=%.8f' % (stat, p))
    # interpret

    if p > alpha:
        print(colored('Same distributions (fail to reject H0)', 'green'))
    else:
        print(colored('Different distributions (reject H0)', 'red'))

In [19]:
interpret_kwtest(s,s2)

Statistics=1997.242, p=0.00000000
Different distributions (reject H0)


In [20]:
from scipy.stats import shapiro
# normality test
# interpret results
def interpret_normaltest(data, alpha=0.05):
    stat, p = shapiro(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))

    if p > alpha:
        print(colored('Sample looks Gaussian (fail to reject H0)', 'green'))
    else:
        print(colored('Sample does not look Gaussian (reject H0)', 'red'))



In [21]:
interpret_normaltest(s)

Statistics=0.999, p=0.162
Sample looks Gaussian (fail to reject H0)


In [22]:
interpret_normaltest(s2)

Statistics=1.000, p=0.999
Sample looks Gaussian (fail to reject H0)


In [23]:
interpret_normaltest(zz)

Statistics=0.236, p=0.000
Sample does not look Gaussian (reject H0)


## Normality Tests

In [24]:
df_mean.head()

,Amostra,RPL32,VGR,VG,Group,Day,DCT_VG,DCT_VGR,DDCT_VG,DDCT_VGR,EXP_DDCT_VG,EXP_DDCT_VGR
0,A1-1,19.731661,33.665260,25.279843,A,1,5.548182,13.933599,2.652499,8.723828,0.159044,0.002365
1,A1-1,19.851553,32.612957,25.306593,A,1,5.455040,12.761404,2.559357,7.551634,0.169651,0.005330
2,A1-2,19.448266,27.550795,24.992273,A,1,5.544006,8.102528,2.648323,2.892758,0.159505,0.134646
3,A1-2,19.733913,27.364021,24.987568,A,1,5.253656,7.630108,2.357972,2.420338,0.195065,0.186812
4,A1-3,18.480774,28.844547,24.236149,A,1,5.755375,10.363774,2.859692,5.154003,0.137768,0.028086


In [25]:
groups = df_mean.Group.unique()
groups

array(['A', 'P', 'B', 'C', 'D'], dtype=object)

In [40]:
days = [1, 4, 8 ]

In [33]:
fields = ['EXP_DDCT_VG', 'EXP_DDCT_VGR']

In [84]:
def interpret_kwtest_1way(a, alpha = 0.05):
    stat, p = stats.kruskal(a[0], a[1], a[2])
    print('Statistics=%.3f, p=%.8f' % (stat, p))
    # interpret

    if p > alpha:
        print(colored('Same distributions (fail to reject H0)', 'green'))
    else:
        print(colored('Different distributions (reject H0)', 'red'))

In [96]:
for f in fields: 
    for g in ['A', 'B', 'C', 'D']: 
        dists = []
        dist_str = ''
        for d in days: 
            dist_str += f'{g}{d} '
            dists.append(df_mean[(df_mean.Day == d) & (df_mean.Group == g)][f].values)
        print(f, g, f'fone_way({dist_str})')
        interpret_kwtest_1way(dists)
        print()

EXP_DDCT_VG A fone_way(A1 A4 A8 )
Statistics=0.573, p=0.75084975
Same distributions (fail to reject H0)

EXP_DDCT_VG B fone_way(B1 B4 B8 )
Statistics=5.509, p=0.06364809
Same distributions (fail to reject H0)

EXP_DDCT_VG C fone_way(C1 C4 C8 )
Statistics=6.538, p=0.03804423
Different distributions (reject H0)

EXP_DDCT_VG D fone_way(D1 D4 D8 )
Statistics=11.661, p=0.00293687
Different distributions (reject H0)

EXP_DDCT_VGR A fone_way(A1 A4 A8 )
Statistics=0.784, p=0.67583057
Same distributions (fail to reject H0)

EXP_DDCT_VGR B fone_way(B1 B4 B8 )
Statistics=1.298, p=0.52250391
Same distributions (fail to reject H0)

EXP_DDCT_VGR C fone_way(C1 C4 C8 )
Statistics=1.556, p=0.45942582
Same distributions (fail to reject H0)

EXP_DDCT_VGR D fone_way(D1 D4 D8 )
Statistics=2.842, p=0.24145971
Same distributions (fail to reject H0)



In [97]:
for f in fields: 
    for d in days: 
        dists = []
        dist_str = ''
        for g in ['A', 'B', 'C', 'D']: 
            dist_str += f'{g}{d} '
            dists.append(df_mean[(df_mean.Day == d) & (df_mean.Group == g)][f].values)
        print(f, d, f'fone_way({dist_str})')
        interpret_kwtest_1way(dists)
        print()

EXP_DDCT_VG 1 fone_way(A1 B1 C1 D1 )
Statistics=7.029, p=0.02975911
Different distributions (reject H0)

EXP_DDCT_VG 4 fone_way(A4 B4 C4 D4 )
Statistics=0.608, p=0.73779183
Same distributions (fail to reject H0)

EXP_DDCT_VG 8 fone_way(A8 B8 C8 D8 )
Statistics=3.591, p=0.16607403
Same distributions (fail to reject H0)

EXP_DDCT_VGR 1 fone_way(A1 B1 C1 D1 )
Statistics=1.263, p=0.53175153
Same distributions (fail to reject H0)

EXP_DDCT_VGR 4 fone_way(A4 B4 C4 D4 )
Statistics=6.877, p=0.03210972
Different distributions (reject H0)

EXP_DDCT_VGR 8 fone_way(A8 B8 C8 D8 )
Statistics=11.556, p=0.00309559
Different distributions (reject H0)



In [88]:
d

8

In [89]:
df_mean[(df_mean.Day == d) & (df_mean.Group == g)]

,Amostra,RPL32,VGR,VG,Group,Day,DCT_VG,DCT_VGR,DDCT_VG,DDCT_VGR,EXP_DDCT_VG,EXP_DDCT_VGR
72,D8-1,21.640908,31.650677,28.051537,D,8,6.410630,10.009769,3.514946,4.799999,0.087477,0.035897
73,D8-1,21.674982,32.221210,27.392739,D,8,5.717757,10.546229,2.822074,5.336458,0.141407,0.024749
74,D8-2,20.920304,32.135807,25.942998,D,8,5.022695,11.215503,2.127011,6.005733,0.228932,0.015563
75,D8-2,21.002150,31.960926,26.077807,D,8,5.075657,10.958776,2.179974,5.749006,0.220680,0.018594
76,D8-3,22.317273,30.891573,30.746798,D,8,8.429525,8.574300,5.533841,3.364529,0.021585,0.097090
77,D8-3,22.333588,31.282587,30.525285,D,8,8.191697,8.948999,5.296014,3.739229,0.025453,0.074882


In [ ]:
df_mean.groupby(['Day', 'Group']).count()

In [50]:
combinations = [ 
    ('A', 'B'),
('A', 'C'),
('A', 'D'),
('B', 'C'),
('B', 'D'),
('C', 'D'),] 

In [51]:
pair_distributions_vg  = {f"{g1}-{g2}-{d}": (
                          df_mean[(df_mean['Group'] == g1) & (df_mean['Day'] == d)]['EXP_DDCT_VG'].values,  
                          df_mean[(df_mean['Group'] == g2) & (df_mean['Day'] == d)]['EXP_DDCT_VGR'].values) for g1, g2 in combinations for d in [1, 4, 8]}
pair_distributions_vgr  = {f"{g1}-{g2}-{d}": (
                          df_mean[(df_mean['Group'] == g1) & (df_mean['Day'] == d)]['EXP_DDCT_VGR'].values, 
                          df_mean[(df_mean['Group'] == g2) & (df_mean['Day'] == d)]['EXP_DDCT_VGR'].values) for g1, g2 in combinations for d in [1, 4, 8]}

## Normality Test for VG 

In [52]:
len(pair_distributions_vg.keys())

18

In [53]:
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})
for k, v in pair_distributions_vg.items():
    print(f'Distributions {k}')
    interpret_normaltest(v[0])
    interpret_normaltest(v[1])

#     plt.hist(v, bins=7)
#     plt.gca().set(ylabel='Frequency')
#     plt.title(k)
#     plt.show()
    print('\n\n')

Distributions A-B-1
Statistics=0.937, p=0.636
Sample looks Gaussian (fail to reject H0)
Statistics=0.817, p=0.082
Sample looks Gaussian (fail to reject H0)



Distributions A-B-4
Statistics=0.722, p=0.011
Sample does not look Gaussian (reject H0)
Statistics=0.859, p=0.187
Sample looks Gaussian (fail to reject H0)



Distributions A-B-8
Statistics=0.928, p=0.568
Sample looks Gaussian (fail to reject H0)
Statistics=0.965, p=0.858
Sample looks Gaussian (fail to reject H0)



Distributions A-C-1
Statistics=0.937, p=0.636
Sample looks Gaussian (fail to reject H0)
Statistics=0.845, p=0.143
Sample looks Gaussian (fail to reject H0)



Distributions A-C-4
Statistics=0.722, p=0.011
Sample does not look Gaussian (reject H0)
Statistics=0.730, p=0.013
Sample does not look Gaussian (reject H0)



Distributions A-C-8
Statistics=0.928, p=0.568
Sample looks Gaussian (fail to reject H0)
Statistics=0.932, p=0.594
Sample looks Gaussian (fail to reject H0)



Distributions A-D-1
Statistics=0.937, p=0.636


## Normality Test for VGR


In [54]:
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})
for k, v in pair_distributions_vgr.items():
    print(f'Distributions {k}')
    interpret_normaltest(v[0])
    interpret_normaltest(v[1])

#     plt.hist(v, bins=7)
#     plt.gca().set(ylabel='Frequency')
#     plt.title(k)
#     plt.show()
    print('\n\n')

Distributions A-B-1
Statistics=0.833, p=0.113
Sample looks Gaussian (fail to reject H0)
Statistics=0.817, p=0.082
Sample looks Gaussian (fail to reject H0)



Distributions A-B-4
Statistics=0.767, p=0.029
Sample does not look Gaussian (reject H0)
Statistics=0.859, p=0.187
Sample looks Gaussian (fail to reject H0)



Distributions A-B-8
Statistics=0.867, p=0.215
Sample looks Gaussian (fail to reject H0)
Statistics=0.965, p=0.858
Sample looks Gaussian (fail to reject H0)



Distributions A-C-1
Statistics=0.833, p=0.113
Sample looks Gaussian (fail to reject H0)
Statistics=0.845, p=0.143
Sample looks Gaussian (fail to reject H0)



Distributions A-C-4
Statistics=0.767, p=0.029
Sample does not look Gaussian (reject H0)
Statistics=0.730, p=0.013
Sample does not look Gaussian (reject H0)



Distributions A-C-8
Statistics=0.867, p=0.215
Sample looks Gaussian (fail to reject H0)
Statistics=0.932, p=0.594
Sample looks Gaussian (fail to reject H0)



Distributions A-D-1
Statistics=0.833, p=0.113


## Comparison Scenarios

### Statistical Tests VG


#### Inter Group VG

In [55]:
for k, v in pair_distributions_vg.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print(k, a, b)
    print('\n\n')

Scenario A-B-1
Statistics=8.308, p=0.00394775
Different distributions (reject H0)
A-B-1 [0.15904438 0.16965117 0.15950539 0.19506515 0.13776756 0.15418375] [0.04114556 0.04489855 0.01862706 0.01621567 0.03799099 0.03889462]



Scenario A-B-4
Statistics=8.308, p=0.00394775
Different distributions (reject H0)
A-B-4 [0.09909119 0.08690795 0.09027376 0.07826293 0.86953017 0.51326163] [0.03123667 0.04780015 0.05100729 0.04762923 0.01663919 0.01073643]



Scenario A-B-8
Statistics=8.308, p=0.00394775
Different distributions (reject H0)
A-B-8 [0.24250009 0.22164061 0.08252701 0.09406522 0.15714524 0.17002302] [0.03810106 0.0458824  0.06887178 0.04881336 0.03292519 0.02397057]



Scenario A-C-1
Statistics=0.923, p=0.33666837
Same distributions (fail to reject H0)
A-C-1 [0.15904438 0.16965117 0.15950539 0.19506515 0.13776756 0.15418375] [0.07742357 0.10109229 0.00927469 0.00725123 0.30028029 0.27430702]



Scenario A-C-4
Statistics=0.103, p=0.74877404
Same distributions (fail to reject H0)
A-C-

### Statistical Tests VGR

#### Inter Group VGR

In [56]:
for k, v in pair_distributions_vgr.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')

Scenario A-B-1
Statistics=0.000, p=1.00000000
Same distributions (fail to reject H0)



Scenario A-B-4
Statistics=0.641, p=0.42333964
Same distributions (fail to reject H0)



Scenario A-B-8
Statistics=0.410, p=0.52183939
Same distributions (fail to reject H0)



Scenario A-C-1
Statistics=0.923, p=0.33666837
Same distributions (fail to reject H0)



Scenario A-C-4
Statistics=2.077, p=0.14954135
Same distributions (fail to reject H0)



Scenario A-C-8
Statistics=8.308, p=0.00394775
Different distributions (reject H0)



Scenario A-D-1
Statistics=0.923, p=0.33666837
Same distributions (fail to reject H0)



Scenario A-D-4
Statistics=0.103, p=0.74877404
Same distributions (fail to reject H0)



Scenario A-D-8
Statistics=0.103, p=0.74877404
Same distributions (fail to reject H0)



Scenario B-C-1
Statistics=0.923, p=0.33666837
Same distributions (fail to reject H0)



Scenario B-C-4
Statistics=7.410, p=0.00648531
Different distributions (reject H0)



Scenario B-C-8
Statistics=8.308, p=0.0